In [1]:
# Imports
from dask_jobqueue import SLURMCluster
from distributed import Client
from BD_cocycles_counterfactual import run as run_cocycles
from BD_SCM_counterfactual_gauss import run as run_flows_gaus
from BD_SCM_counterfactual_laplace import run as run_flows_laplace
from BD_SCM_counterfactual_tailadaptive import run as run_flows_tailadaptive

In [2]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [3]:
# Submitting jobs
n = 10000
ntrial = 20
base = "Normal"
futures = []
for seed in range(ntrial):
    f1 = client.submit(run_cocycles,seed,n,base = base)
    f2 = client.submit(run_flows_gaus,seed,n, base = base)
    f3 = client.submit(run_flows_laplace,seed,n, base = base)
    f4 = client.submit(run_flows_tailadaptive,seed,n, base = base)
    futures += [f1,f2,f3,f4]

In [4]:
futures

[<Future: pending, key: run-619447e665545ef153768b666d960889>,
 <Future: pending, key: run-4ba5d272c780814078e466f5cf15d83d>,
 <Future: pending, key: run-8994b233047d7bc93c062973cfc1c402>,
 <Future: pending, key: run-c6a2408c7161780e8538fc4029d3ad86>,
 <Future: pending, key: run-b783085913e5d5aaa01d72369ffd897c>,
 <Future: pending, key: run-2c24399cfffcfeba6a9b6ee8ea0d94f4>,
 <Future: pending, key: run-114a3595926722fe3987c4920be43283>,
 <Future: pending, key: run-880fb23e1614813ccbb56e09ffe60f1f>,
 <Future: pending, key: run-fffb2cdd998e57c7bdb2e786aa05913e>,
 <Future: pending, key: run-0636995b96267ce467c183db57d4ac03>,
 <Future: pending, key: run-7c178d0b5d8b0f6761a0d1e4b7f055b9>,
 <Future: pending, key: run-7157b2325d350f5ee89938405b55ed3c>,
 <Future: pending, key: run-7aef5eb00d5002609ec02a6050511593>,
 <Future: pending, key: run-7274d6a583e6db88ddc19388331749f9>,
 <Future: pending, key: run-7cfce975e7c7853c3b9f092fb3616a70>,
 <Future: pending, key: run-a795aa3e8985ea4e0917254f15a

In [5]:
# Getting results
results = client.gather(futures)

In [6]:
# Closing client
client.close()
cluster.close()

2025-04-22 21:50:50,251 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
import torch
torch.save(f = "BD_results_new_trials={0}_n={1}_base={2}.pt".format(ntrial,n,base), obj = results)